In [210]:
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import numpy as np

In [212]:
# Load the dataset.
data = pd.read_csv("Electric_cars_dataset.csv")

In [214]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64353 entries, 0 to 64352
Data columns (total 18 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 64353 non-null  object 
 1   VIN (1-10)                                         64353 non-null  object 
 2   County                                             64349 non-null  object 
 3   City                                               64344 non-null  object 
 4   State                                              64342 non-null  object 
 5   ZIP Code                                           64347 non-null  float64
 6   Model Year                                         64346 non-null  float64
 7   Make                                               64349 non-null  object 
 8   Model                                              64340 non-null  object 
 9   Electr

In [215]:
data.head()

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k)
0,EV33174,5YJ3E1EC6L,Snohomish,LYNNWOOD,WA,98037.0,2020.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,32.0,109821694,POINT (-122.287614 47.83874),PUGET SOUND ENERGY INC,50
1,EV40247,JN1AZ0CP8B,Skagit,BELLINGHAM,WA,98229.0,2011.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,73,0,40.0,137375528,POINT (-122.414936 48.709388),PUGET SOUND ENERGY INC,15
2,EV12248,WBY1Z2C56F,Pierce,TACOMA,WA,98422.0,2015.0,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81,0,27.0,150627382,POINT (-122.396286 47.293138),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,18
3,EV55713,1G1RD6E44D,King,REDMOND,WA,98053.0,2013.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,45.0,258766301,POINT (-122.024951 47.670286),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),33.9
4,EV28799,1G1FY6S05K,Pierce,PUYALLUP,WA,98375.0,2019.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,25.0,296998138,POINT (-122.321062 47.103797),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,41.78


In [216]:
# Drop the unnecessary columns
columns_to_drop = ["ID", "VIN (1-10)", "County", "City", "State", "ZIP Code", "Legislative District", "DOL Vehicle ID"]
data = data.drop(columns=columns_to_drop)

In [220]:
# Display basic statistics.
stats = data.describe()
print(stats)

         Model Year  Electric Range      Base MSRP
count  64346.000000    64353.000000   64353.000000
mean    2018.186212      106.948985    2524.990754
std        2.726742      104.093919   12402.895104
min     1993.000000        0.000000       0.000000
25%     2017.000000       14.000000       0.000000
50%     2018.000000       73.000000       0.000000
75%     2021.000000      215.000000       0.000000
max     2022.000000      337.000000  845000.000000


In [221]:
# Display the dataset
data

,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Vehicle Location,Electric Utility,Expected Price ($1k)
0,2020.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,POINT (-122.287614 47.83874),PUGET SOUND ENERGY INC,50
1,2011.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,73,0,POINT (-122.414936 48.709388),PUGET SOUND ENERGY INC,15
2,2015.0,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81,0,POINT (-122.396286 47.293138),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,18
3,2013.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,POINT (-122.024951 47.670286),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),33.9
4,2019.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,POINT (-122.321062 47.103797),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,41.78
...,...,...,...,...,...,...,...,...,...,...
64348,2020.0,KIA,NIRO,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,239,0,POINT (-122.30033 47.585339),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),33
64349,2013.0,TOYOTA,PRIUS PLUG-IN,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,6,0,POINT (-122.443211 47.252172),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,13.3
64350,2018.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,POINT (-122.367721 47.639264),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),22.857
64351,2017.0,PORSCHE,CAYENNE,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,14,0,POINT (-122.297534 47.685291),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),45.7


In [224]:
# Create and display the profiling report
profile = ProfileReport(data, title="Electric Vehicle Data Profiling Report")
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [225]:
# Check for missing values
data.isnull().sum()

Model Year                                             7
Make                                                   4
Model                                                 13
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                         0
Base MSRP                                              0
Vehicle Location                                     510
Electric Utility                                     722
Expected Price ($1k)                                   0
dtype: int64

In [184]:
# Separate numerical and categorical columns
numerical_cols = data.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = data.select_dtypes(include=["object", "category"]).columns

In [185]:
# Fill missing values for numerical columns with the mean
for col in numerical_cols:
    mean = stats[col].values[1]
    print(f"The mean of {col} is {mean}")
    data[col] = data[col].fillna(mean)

The mean of Model Year is 2018.1862120411524
The mean of Electric Range is 106.94898450732677
The mean of Base MSRP is 2524.990754121797


In [186]:
# Calculate the mode for each categorical column
modes = {}
for col in categorical_cols:
    modes[col] = data[col].mode()[0]

# Fill the missing values in each categorical column using its mode
for col in categorical_cols: 
    data[col].fillna(modes[col], inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_19072\3042116607.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(modes[col], inplace=True)


In [187]:
# Display the number of duplicate rows
num_duplicates = data.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 40251


In [188]:
# Remove duplicate rows
data = data.drop_duplicates()

In [189]:
# Display the new shape of the dataset
print(f"New shape of the dataset: {data.shape}")# Remove duplicate rows
data = data.drop_duplicates()

New shape of the dataset: (24102, 10)


In [190]:
# Display the new shape of the dataset
print(f"New shape of the dataset: {data.shape}")

New shape of the dataset: (24102, 10)


In [191]:
#Retrieving and Saving all categorical data into a variable
categories = data.select_dtypes(include="object").columns
categories

Index(['Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Vehicle Location',
       'Electric Utility', 'Expected Price ($1k)'],
      dtype='object')

In [192]:
encoder = LabelEncoder() #instantiate the label encoder class

for col in categories: 
    print(f"Encoding {col}") 
    data.loc[:, col] = encoder.fit_transform(data[col])

Encoding Make
Encoding Model
Encoding Electric Vehicle Type
Encoding Clean Alternative Fuel Vehicle (CAFV) Eligibility
Encoding Vehicle Location
Encoding Electric Utility
Encoding Expected Price ($1k)


In [193]:
data_sample = data.sample(n = 5000)

data_sample

,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Vehicle Location,Electric Utility,Expected Price ($1k)
22993,2020.0,24,93,0,0,203,0,361,52,14
9044,2017.0,0,8,1,2,16,0,389,67,78
60494,2016.0,26,37,0,0,68,0,511,21,29
17560,2022.0,3,99,1,0,30,0,287,66,174
1766,2019.0,28,58,0,0,270,0,360,66,196
...,...,...,...,...,...,...,...,...,...,...
2539,2021.0,10,56,0,1,0,0,68,4,138
34668,2019.0,15,61,0,0,239,0,463,32,78
4535,2015.0,22,53,0,0,84,0,442,65,78
8354,2014.0,22,53,0,0,84,0,220,66,49


In [194]:
features = data_sample.drop("Expected Price ($1k)", axis=1)
label = data_sample["Expected Price ($1k)"]

features

,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Vehicle Location,Electric Utility
22993,2020.0,24,93,0,0,203,0,361,52
9044,2017.0,0,8,1,2,16,0,389,67
60494,2016.0,26,37,0,0,68,0,511,21
17560,2022.0,3,99,1,0,30,0,287,66
1766,2019.0,28,58,0,0,270,0,360,66
...,...,...,...,...,...,...,...,...,...
2539,2021.0,10,56,0,1,0,0,68,4
34668,2019.0,15,61,0,0,239,0,463,32
4535,2015.0,22,53,0,0,84,0,442,65
8354,2014.0,22,53,0,0,84,0,220,66


In [153]:
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.2)

In [164]:
model = SVR(kernel = "rbf")
model.fit(x_train, y_train)

SVR()

In [166]:
# Model preediction.

model_prediction = model.predict(x_test)
model_prediction

array([116.8653288 , 101.8754626 , 101.92790283, 101.88285556,
       101.88368551, 101.9480255 , 101.87669287, 101.86720772,
       101.88396314, 101.89856865, 101.90623758, 101.86767018,
       101.88408927, 101.8952151 , 101.8833714 , 101.97044672,
       101.9073639 , 101.86572546, 101.91671394, 101.86988847,
       101.99350044, 101.96082948, 101.88042854, 101.90571864,
       112.82202779,  68.10270795, 101.89035321, 101.86163576,
       101.97279607, 101.85945993, 101.94249053, 101.89358858,
       101.87063435, 101.92166087, 101.88434537, 101.89914475,
       101.87133475, 101.88134827, 101.90584844, 101.88606041,
       101.93573836, 101.93015568, 101.9032597 , 101.88652989,
       101.8620865 , 101.97960706, 101.86435415, 101.90953343,
        55.02879412, 101.91541766, 101.8915662 , 101.87024887,
       101.91055288, 101.96533443, 101.88388156,  58.58774178,
       101.90536624, 101.88210703, 101.9763977 , 101.92968963,
       101.92122749, 101.99244487, 101.89010275, 101.92

In [208]:
# Evaluating the regression model
mse = mean_squared_error(y_test, model_prediction)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, model_prediction)
r2 = r2_score(y_test, model_prediction)

print(f"The Mean Squared Error: {mse:.2f}")
print(f"The Root Mean Squared Error is {rmse}")
print(f"The Mean Absolute Error is {mae:.2f}")
print(f"The R^2 Score is {r2:.2f}")

The Mean Squared Error: 2944.81
The Root Mean Squared Error is 54.26610544087861
The Mean Absolute Error is 46.59
The R^2 Score is 0.03


In [226]:
model_r = RandomForestRegressor()
model_r = model_r.fit(x_train, y_train)

In [230]:
model_r_pred = model_r.predict(x_test)

model_r_pred

array([132.11, 140.  , 106.  , 124.  , 189.  , 181.  ,  40.  , 175.  ,
       140.  , 161.  ,  37.72, 150.52, 124.  , 132.  ,  43.  , 205.  ,
        40.  ,  77.38,  84.  ,  51.  , 136.  , 136.  ,  77.  , 114.07,
        54.  ,  68.  ,  52.05,  25.  , 186.  , 108.66, 107.  ,  44.  ,
        36.58,  48.45,  70.62,  24.85, 122.  ,  52.  ,  80.47, 129.61,
       118.  ,  80.09,  49.  , 183.63, 175.  , 186.  , 191.  ,  38.  ,
        51.  ,  25.47, 123.  , 117.  ,  83.94, 176.  ,  40.46,  33.  ,
       188.  , 175.  , 205.  ,  51.21,  49.  , 103.  , 149.  ,  49.  ,
       176.  , 132.  ,  69.  ,  27.  , 196.  , 110.  , 175.  ,  38.  ,
        42.  , 181.  , 170.84,  84.  , 188.  ,   6.  , 103.  ,  60.18,
       152.94, 106.  ,  83.12,  52.  ,  27.  ,  87.  , 170.  , 132.  ,
       188.  ,  99.  , 114.  , 196.  , 175.  ,  99.  ,  31.96, 188.  ,
        99.  ,  87.  ,  65.  ,  52.3 , 185.  , 139.  , 132.  ,  77.  ,
       102.01,  75.  ,  49.  ,  60.18,  49.  , 103.  , 100.  , 131.  ,
      

In [232]:
# Evaluating the regression model
mse = mean_squared_error(y_test, model_r_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, model_r_pred)
r2 = r2_score(y_test, model_r_pred)

print(f"The Mean Squared Error: {mse:.2f}")
print(f"The Root Mean Squared Error is {rmse}")
print(f"The Mean Absolute Error is {mae:.2f}")
print(f"The R^2 Score is {r2:.2f}")

The Mean Squared Error: 43.24
The Root Mean Squared Error is 6.575931029139524
The Mean Absolute Error is 1.05
The R^2 Score is 0.99
